# Importing Data

In [1]:
import pandas as pd

In [3]:
census = pd.read_csv('census_data.csv')

In [4]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
# Fixing the class column to 0 - 1
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [8]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [10]:
census.head(10)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
5,37,Private,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0
6,49,Private,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,0
7,52,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
8,31,Private,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,1
9,42,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,1


# Splitting the dataset

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_data = census.drop('income_bracket', axis=1)

In [13]:
y_labels = census['income_bracket']

In [14]:
X_train,X_test,y_train,y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=0)

# Creating TensorFlow Columns

In [15]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [16]:
import tensorflow as tf

In [20]:
# Create features with categorical values
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender',['Female','Male'])
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass',hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=1000)

In [22]:
# Create features with continiuous values
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

In [23]:
feat_cols = [gender,occupation,relationship,education,workclass,native_country,marital_status,age,education_num,
            capital_gain,capital_loss,hours_per_week]

# Create Input Function

In [27]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)

# Create Model

In [25]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmp2ehjanpv', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_tf_random_seed': 1}


# Train the Model

In [26]:
model.train(input_fn=input_func, steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp2ehjanpv/model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 151.796
INFO:tensorflow:loss = 624.356, step = 101 (0.652 sec)
INFO:tensorflow:global_step/sec: 152.966
INFO:tensorflow:loss = 23.1452, step = 201 (0.654 sec)
INFO:tensorflow:global_step/sec: 171.268
INFO:tensorflow:loss = 110.136, step = 301 (0.587 sec)
INFO:tensorflow:global_step/sec: 135.532
INFO:tensorflow:loss = 284.829, step = 401 (0.732 sec)
INFO:tensorflow:global_step/sec: 158.584
INFO:tensorflow:loss = 35.8355, step = 501 (0.633 sec)
INFO:tensorflow:global_step/sec: 153.462
INFO:tensorflow:loss = 51.5061, step = 601 (0.649 sec)
INFO:tensorflow:global_step/sec: 159.129
INFO:tensorflow:loss = 59.2957, step = 701 (0.631 sec)
INFO:tensorflow:global_step/sec: 158.623
INFO:tensorflow:loss = 278.111, step = 801 (0.627 sec)
INFO:tensorflow:global_step/sec: 180.573
INFO:tensorflow:loss = 41.2846,

INFO:tensorflow:loss = 29.3809, step = 8401 (0.582 sec)
INFO:tensorflow:global_step/sec: 183.727
INFO:tensorflow:loss = 54.6232, step = 8501 (0.544 sec)
INFO:tensorflow:global_step/sec: 164.625
INFO:tensorflow:loss = 33.5359, step = 8601 (0.608 sec)
INFO:tensorflow:global_step/sec: 170.766
INFO:tensorflow:loss = 63.4698, step = 8701 (0.585 sec)
INFO:tensorflow:global_step/sec: 165.374
INFO:tensorflow:loss = 73.9921, step = 8801 (0.606 sec)
INFO:tensorflow:global_step/sec: 158.272
INFO:tensorflow:loss = 31.4572, step = 8901 (0.631 sec)
INFO:tensorflow:global_step/sec: 158.777
INFO:tensorflow:loss = 40.9903, step = 9001 (0.627 sec)
INFO:tensorflow:global_step/sec: 178.136
INFO:tensorflow:loss = 30.3869, step = 9101 (0.564 sec)
INFO:tensorflow:global_step/sec: 177.192
INFO:tensorflow:loss = 31.2211, step = 9201 (0.565 sec)
INFO:tensorflow:global_step/sec: 184.17
INFO:tensorflow:loss = 44.1439, step = 9301 (0.540 sec)
INFO:tensorflow:global_step/sec: 183.934
INFO:tensorflow:loss = 311.383,

# Evaluation Phase

In [28]:
pred_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), num_epochs=1, shuffle=False)

In [29]:
pred_gen = model.predict(input_fn=pred_func)

In [30]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /tmp/tmp2ehjanpv/model.ckpt-10000


In [31]:
predictions

[{'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.01573097], dtype=float32),
  'logits': array([-4.13626814], dtype=float32),
  'probabilities': array([ 0.98426902,  0.01573097], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.05191635], dtype=float32),
  'logits': array([-2.904809], dtype=float32),
  'probabilities': array([ 0.94808364,  0.05191635], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.30038846], dtype=float32),
  'logits': array([-0.84544879], dtype=float32),
  'probabilities': array([ 0.69961154,  0.30038846], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.58081442], dtype=float32),
  'logits': array([ 0.32611752], dtype=float32),
  'probabilities': array([ 0.41918558,  0.58081442], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array(

In [35]:
final_preds = [pred['class_ids'][0] for pred in predictions]

In [36]:
final_preds

[0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [37]:
from sklearn.metrics import classification_report, confusion_matrix

In [38]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.89      0.91      0.90      7407
          1       0.69      0.64      0.66      2362

avg / total       0.84      0.84      0.84      9769

